In [1]:
!pip3 install peft==0.7.1
!pip3 install trl==0.7.4
!pip3 install transformer==4.36.2

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement transformer==4.36.2 (from versions: none)
ERROR: No matching distribution found for transformer==4.36.2


In [2]:
import transformers
import trl
import os
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/home/koala/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-25 23:31:47.067677: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-25 23:31:47.097825: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-25 23:31:47.666721: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/ko

### 1. Alpaca dataset

In [3]:
# Step1: load the json file

from datasets import Dataset
import json

json_file_path = "alpaca_data.json"

alpaca_dataset = Dataset.from_json(json_file_path)

In [4]:
alpaca_dataset

Dataset({
    features: ['input', 'output', 'instruction'],
    num_rows: 52002
})

In [5]:
from datasets import load_dataset
alpaca_eval_dataset = load_dataset("tatsu-lab/alpaca_eval", split='eval')
alpaca_eval_dataset

/home/koala/.local/lib/python3.11/site-packages/datasets/load.py:1429: FutureWarning: The repository for tatsu-lab/alpaca_eval contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/tatsu-lab/alpaca_eval
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Dataset({
    features: ['instruction', 'output', 'generator', 'dataset'],
    num_rows: 805
})

### 2. Model Training

In [6]:
# Step1: Load the model & Tokenizer

from transformers import AutoModelForCausalLM, AutoTokenizer
model_name_or_path = "distilgpt2"
model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path, device_map = 'auto')
tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

In [7]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['instruction'])):
        text = f"### Question: {example['instruction'][i]}\n ### Answer: {example['output'][i]}"
        output_texts.append(text)
    return output_texts

#check instruction-prompt
formatting_prompts_func(alpaca_dataset[:2])

['### Question: Give three tips for staying healthy.\n ### Answer: 1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.',
 '### Question: What are the three primary colors?\n ### Answer: The three primary colors are red, blue, and yellow.']

In [8]:
# use the DataCollatorForCompletionOnlyLM to train your model on the generated prompts only
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
response_template = " ### Answer:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)
collator

DataCollatorForCompletionOnlyLM(tokenizer=GPT2TokenizerFast(name_or_path='distilgpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}, mlm=False, mlm_probability=0.15, pad_to_multiple_of=None, tf_experimental_compile=False, return_tensors='pt')

In [9]:
max_seq_length = min(tokenizer.model_max_length, 64)
max_seq_length

64

In [10]:
# Step2: Define the Trainer
trainer = SFTTrainer(
    model,
    train_dataset=alpaca_dataset.select(range(1000)),
    eval_dataset=alpaca_eval_dataset.select(range(50)),
    formatting_func=formatting_prompts_func,
    data_collator=collator,
    max_seq_length=max_seq_length
)

trainer.train() 


100%|██████████| 375/375 [00:40<00:00,  9.33it/s]

{'train_runtime': 40.1778, 'train_samples_per_second': 74.668, 'train_steps_per_second': 9.334, 'train_loss': 2.618673177083333, 'epoch': 3.0}


TrainOutput(global_step=375, training_loss=2.618673177083333, metrics={'train_runtime': 40.1778, 'train_samples_per_second': 74.668, 'train_steps_per_second': 9.334, 'train_loss': 2.618673177083333, 'epoch': 3.0})

### 3. Model Evaluation

In [11]:
trainer.evaluate()

100%|██████████| 7/7 [00:00<00:00, 47.27it/s]


{'eval_loss': 2.6091973781585693,
 'eval_runtime': 0.1796,
 'eval_samples_per_second': 278.358,
 'eval_steps_per_second': 38.97,
 'epoch': 3.0}

In [24]:
# Encode input text
input_text = "Explain me about python programming"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

# Generate output
output = model.generate(input_ids, max_length=256, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95, temperature=0.7)

# Decode and print the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated text:\n", generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated text:
 Explain me about python programming. It’s a powerful and powerful programming language that can be used to solve complex problems in a variety of different languages and languages. Python is an open-source language, which allows programmers to quickly and easily implement complex programming languages without having to rely on proprietary software or hardware to do the work. Additionally, it is easy to use and has the potential to revolutionize the way programming is written and used in many languages, such as Python, C++, and C#. The Python language is also widely used for developing applications and applications, allowing developers to create and maintain applications that are easily accessible to the general public. Furthermore, the language has a wide range of features, including support for JavaScript and JavaScript, as well as a number of other features. For example, there are many features that make it easy for developers and developers alike to learn and use Py

In [28]:
import pickle
model_path = "./app/models/trained_model.pth"

torch.save(model.state_dict(), model_path)